In [1]:
import time

from getpass import getpass

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

import os
 
import pandas as pd

import pathlib
import os
from os.path import join

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
driver.get('https://www.investing.com/equities/amazon-com-inc-news')
time.sleep(1.5)

In [4]:
# delete the "Accept cookies pop-up notification":

signin = driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]')
signin.click()
time.sleep(2)

In [7]:
# delete the "Language":

signin = driver.find_element(By.XPATH,'''//*[@onclick="postponeEdition(14, 'No Thanks')"]''')

signin.click()

time.sleep(1.5)

In [8]:
# delete the ad:

signin = driver.find_element(By.CLASS_NAME, 'dfpCloseBtn')

signin.click()

time.sleep(1.5)

In [9]:
driver.execute_script("document.body.style.zoom='55%'")

In [11]:
art_raw = driver.find_elements(By.CSS_SELECTOR,"div[class^='mediumTitle1']")

def scrapper(numb_art = len(art_raw)): ## add pages

    # For scraper reasons it's required to duplicate the numb_j as it retrieves 2 times the same position:
    numb_art = numb_art *2
    
    # empty list for saving the job names , link and extra info: 
    art_list = []

    # reduce the page size in order to be able to find the name of the job in the right session 
    driver.execute_script("document.body.style.zoom='55%'")

    # all jobs in the page
    art_raw = driver.find_elements(By.CSS_SELECTOR,"div[class^='mediumTitle1']") 

    for art_idx in range(numb_art):
        if art_idx % 2 == 0:
            # get the job link
            ref = art_raw[art_idx].get_attribute('href') 
            time.sleep(2)

            ## let's click on the job post ##            

            # get the job name with the .text method
            art_name = driver.find_element(By.CSS_SELECTOR,"a[class^='title']").text
            time.sleep(2.2)            

            # get company name: 

            art_details = "amazon"

            # populate list:
            art_idx_list = [ref, art_name , art_details]
            time.sleep(3)

            # page.send_keys(Keys.PAGE_DOWN)

            art_list.append(art_idx_list)

            print(f"Collected job : {art_name} for company: {art_details}")

    #Create dataframe:
    art_dataframe = pd.DataFrame(art_list, 
                                 columns = ["article_link", "title", "company name"]
                                ).drop_duplicates()


    #Save dataframe in excel file to later use our job 
    art_dataframe.to_excel(pathlib.Path().joinpath('scraped_articles.xlsx'), 
                           sheet_name='investing',
                           index= False)
    
    return art_dataframe